In [1]:
import nltk 
from nltk.stem import WordNetLemmatizer 
from glob import glob as gb
from random import sample
from bs4 import BeautifulSoup
import json
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import random
import pandas as pd
import math
import spacy
import subprocess
import concurrent.futures
import string 
from utils.functions import *
import numpy as np

In [2]:
data_path = "/media/ruben/Elements/PhD/data/hansard/lemmatized_pm"
list_files = gb(data_path + "/*")

In [3]:
## Subset paragraphs that mention bureaucracy with grep, into one dataframe
path = "/media/ruben/Elements/PhD/data/hansard/lemmatized_pm/*"
grp = "egrep -iE '" + "|".join(["bureaucracy"]) + "' " + path
output = subprocess.check_output(grp,shell=True).decode('utf-8')
output = [l.split('\t') for l in output.split('\n')]
data = pd.DataFrame(output)
data.columns = "id text topic_id topic_name scene_id scene_name speech_id speech_member_ref speech_member_ref_name speech_member_party num_char text_lemmatized".split(' ')
data = data[data['text'].astype(str) != "text"].reset_index(drop=True)
data = data.dropna().reset_index(drop=True)

In [27]:
## Extract 3-sentence windows, add speaker-id and id 
#  I take one sentence left, and one sentence right to the sentence containing the keyword.

def sentences2windows(sentences,ind_):
    left = ind_- 1
    right = ind_ + 2
    if left < 0:
        left = 0
    if right > len(sentences):
        right = len(sentences)
    return " ".join(sentences[left:right])

windows = []

for c,t in enumerate(data['text_lemmatized']):
    sentences = nltk.sent_tokenize(t)
    indices = [c for c,s in enumerate(sentences) if "bureaucracy" in set(s.split(' '))]
    
    if len(indices) == 0:
        continue 
    elif len(indices) == 1:
        windows.append([data['id'][c],data['speech_member_ref'][c],data['speech_member_party'][c],sentences2windows(sentences,indices[0])])
    elif len(indices) > 1:
        if abs(indices[0]-indices[1]) == 1:
            windows.append([data['id'][c],data['speech_member_ref'][c],data['speech_member_party'][c],sentences2windows(sentences,indices[0])])
        else:
            print(c,indices)
df = pd.DataFrame(windows,columns=["id","member-ref","member-party","window"])
df['id'] = [df['member-ref'][c][5:] + '-' + os.path.split(x)[-1].split(':')[1][10:] + "-" + df['member-party'][c].replace(' ','_').lower() for c,x in enumerate(df['id'])]
df = df[["id","window"]]

1457 [0, 2]
2019 [0, 2]
2302 [0, 2]
2789 [0, 2]


In [5]:
df.to_csv('/media/ruben/OSDisk/Users/ruben.ros/Documents/GitHub/CrisisBureaucracy/data/bureaucracy-sentences-full.tsv',sep='\t',index=False)

In [23]:
with open('/media/ruben/OSDisk/Users/ruben.ros/Documents/GitHub/CrisisBureaucracy/data/bureaucracy-sentences-sample-before1970.html','w') as f:
    ht = df_sample.to_html(bold_rows=True,table_id="table-content",index_names=False,index=False)
    ht = ht.replace(' bureaucracy ',' <b>bureaucracy</b> ')
    ht = '<html>' + '<link rel="stylesheet" href="css/style.css" /> <link href="https://fonts.googleapis.com/css2?family=Crimson+Text&family=Lato&family=Raleway&display=swap" rel="stylesheet">' + ht + '</html>'
    f.write(ht)

In [41]:
msk = np.random.rand(len(df)) < 0.85

In [42]:
training_data = df[~msk].reset_index(drop=True)
other_data = df[msk].reset_index(drop=True)

In [43]:
training_data_full = training_data
training_data_doccano = pd.DataFrame(training_data_full.iloc[:,1])
other_data_full = other_data
other_data_doccano = pd.DataFrame(other_data_full.iloc[:,1])

In [46]:
base_path = '/media/ruben/OSDisk/Users/ruben.ros/Documents/GitHub/CrisisBureaucracy/data/classifier'
training_data_full.to_csv(os.path.join(base_path,"training_data_full.csv"),index=False,sep='\t')
training_data_doccano.to_csv(os.path.join(base_path,"training_data_doccano.txt"),index=False,sep='\t',header=False)
other_data_full.to_csv(os.path.join(base_path,"other_data_full.csv"),index=False,sep='\t')
other_data_doccano.to_csv(os.path.join(base_path,"other_data_doccano.txt"),index=False,sep='\t',header=False)

In [22]:
## Subset for Round 2 - TagToc

df = pd.read_csv('~/Documents/GitHub/CrisisBureaucracy/data/classifier/bureaucracy-sentences-full.tsv',sep='\t').dropna()
sample_ = df.sample(750)

In [23]:
for i,r in sample_.iterrows():

    with open('/home/ruben/Documents/GitHub/CrisisBureaucracy/data/classifier/annotation-round-2/' + r['id'] + ".txt",'w') as f:
         f.write(r['window'])